In [ ]:
%%writefile simd_divergence.c
#include <immintrin.h>
#include <stdio.h>

int main() {
    // 8 integers
    __m256i values = _mm256_setr_epi32(1, 5, 10, 3, 8, 12, 2, 15);

    // IF value > 5: result = value * 2
    // ELSE: result = value + 10

    // MANUAL branching - painful!
    __m256i threshold = _mm256_set1_epi32(5);
    __m256i mask = _mm256_cmpgt_epi32(values, threshold);  // Create mask

    __m256i if_true = _mm256_slli_epi32(values, 1);      // value * 2
    __m256i if_false = _mm256_add_epi32(values, _mm256_set1_epi32(10)); // value + 10

    // Blend based on mask (manual if/else!)
    __m256i result = _mm256_blendv_epi8(if_false, if_true, mask);

    int out[8];
    _mm256_storeu_si256((__m256i*)out, result);

    printf("SIMD (manual masking):\n");
    int vals[8]; _mm256_storeu_si256((__m256i*)vals, values);
    for (int i = 0; i < 8; i++) {
        printf("%d -> %d\n", vals[i], out[i]);
    }

    return 0;
}

Writing simd_divergence.c


In [ ]:
!gcc -O3 -Wall -Wextra -mavx2  -o simd_divergence simd_divergence.c
!./simd_divergence

SIMD (manual masking):
1 -> 11
5 -> 15
10 -> 20
3 -> 13
8 -> 16
12 -> 24
2 -> 12
15 -> 30
